# Thermodynamic Relations

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb.FuldeFerrelState import FFState
from mmf_hfb import homogeneous
import itertools
tf.MAX_DIVISION = 500

### Energy Density for 1D free Fermi Gas($\Delta=0$)
$$
\mathcal{E}=\frac{\sqrt{2m}}{\pi\hbar}\frac{E_F^{3/2}}{3}
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{\sqrt{2m}}{\pi\hbar}\frac{\mu_a^{3/2}+\mu_b^{3/2}}{3} \\
&=\frac{\sqrt{2m}}{\pi\hbar}\frac{(\mu+d\mu)^{3/2}+(\mu-d\mu)^{3/2}}{3}
\end{align}

### Energy Density for 2D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{m}{4\pi\hbar^2}E_F^2\\
n=\frac{m}{\pi\hbar^2}E_F\\
\mathcal{E}=\frac{1}{2}nE_F
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{m}{4\pi\hbar^2}(\mu_a^2+\mu_b^2)\\
&=\frac{m}{4\pi\hbar^2}\left((\mu+d\mu)^2+(\mu-d\mu)^2\right)\\
\end{align}

### Energy Density for 3D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{\hbar^2}{10m\pi^2}k_F^5=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}E_F^{5/2}\\
n=\frac{k_F^3}{3\pi^2}
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}(\mu_a^{5/2}+\mu_b^{5/2})\\
&=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}\left((\mu+d\mu)^{5/2}+(\mu+d\mu)^{5/2}\right)\\
\end{align}

## Test Code
* The $\Delta_0$ is set to unitary, which may be not good for large $mu$ and $dmu$

In [ ]:
def test_Thermodynamic(mu, dmu,delta0=1, dim=1, k_c=100, q=0, dq=0):
    print(f"mu={mu}\tdmu={dmu}\tkc={k_c}\tq={q}\tdq={dq}\tdim={dim}")
    dx = 1e-6
    ff = FFState(dmu=dmu, mu=mu, delta=delta0, dim=dim, k_c=k_c, fix_g=False)
    
    def get_P(mu, dmu):
        delta = ff.solve(mu=mu, dmu=dmu, q=q, dq=dq)
        print(f"Delta={delta}")
        return ff.get_pressure(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)

    def get_E_n(mu, dmu):
        E = ff.get_energy_density(mu=mu, dmu=dmu, q=q, dq=dq)
        n = sum(ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq))
        return E, n

    def get_ns(mu, dmu):
        return ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
    if dim == 1:
        energy_density = np.sqrt(2)/np.pi *( (mu+dmu)**1.5 + (mu-dmu)**1.5)/3.0
    elif dim == 2:
        energy_density = ((mu+dmu)**2 + (mu-dmu)**2)/4.0/np.pi
    elif dim == 3:
        energy_density = ((mu+dmu)**2.5 + (mu-dmu)**2.5)*2.0**1.5/10.0/np.pi**2
    print(f"Free Energy Density {dim}D:{energy_density}")    
    E1, n1 = get_E_n(mu=mu+dx, dmu=dmu)
    E0, n0 = get_E_n(mu=mu-dx, dmu=dmu)
    print(f"E1={E1}\tE0={E0}")
    n_p = (get_P(mu+dx, dmu) - get_P(mu-dx, dmu))/2/dx
    n_a, n_b = get_ns(mu, dmu)
    n_a_ = (get_P(mu+dx/2, dmu+dx/2) - get_P(mu-dx/2, dmu - dx/2))/2/dx
    n_b_ = (get_P(mu+dx/2, dmu-dx/2) - get_P(mu-dx/2, dmu + dx/2))/2/dx
    print(f"n_a={n_a.n}\tNumerical  n_a={n_a_.n}")
    print(f"n_b={n_b.n}\tNumerical  n_b={n_b_.n}")
    print(f"n_p={n_a.n+n_b.n}\tNumerical  n_p={n_p.n}")
    print(f"mu={mu}\tNumerical mu={((E1-E0)/(n1-n0)).n}")
    assert np.allclose(n_a.n, n_a_.n, rtol=1e-4)
    assert np.allclose(n_b.n, n_b_.n, rtol=1e-4)
    assert np.allclose(n_p.n, (n_a+n_b).n, rtol=1e-4)
    assert np.allclose(mu,((E1-E0)/(n1-n0)).n, rtol=1e-2)
    
def homogeneous_Density(mu, dmu, delta, dim):
    """return density,used for reference"""
    return homogeneous.Homogeneous(dim = dim).get_ns(mus_eff=(mu+dmu, mu-dmu), delta=delta, N_twist=12)

## 1D Cases 

In [ ]:
homogeneous_Density(mu=5, dmu=0.5, delta=1, dim=1)

### Good accurary for $\mu_+=\frac{\partial E}{\partial n_+}$

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=0, dq=0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=.5, dq=1.0, dim=1, delta0=1)

### Medium accurary

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=2.0, dq=1.0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=.5, k_c=200, q=1.0, dq=1.0, dim=1, delta0=1)

## Poor accurary for  $\mu_+=\frac{\partial E}{\partial n_+}$

In [ ]:
homogeneous_Density(mu=5, dmu=2.5, delta=5, dim=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=0, dq=0, dim=1, delta0=1)

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=500, q=0, dq=0, dim=1, delta0=5)

### $q$ and $dq$ do change the density

In [ ]:
test_Thermodynamic(mu=5, dmu=2.5, k_c=100, q=.5, dq=1.0, dim=1, delta0=1)

# 2D Cases

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=2)

### Good accuracy

In [ ]:
test_Thermodynamic(mu=10, dmu=0.64, dim=2, q=0, dq=0, delta0=1, k_c=200)

In [ ]:
test_Thermodynamic(mu=10, dmu=0.64, dim=2, q=1.0, dq=0.5, delta0=1, k_c=200)

### A local maximum in pressure with $dq \ne 0$

In [ ]:
qs = [0.5]#np.linspace(0,3,4) # q does not change the pressure, you may use q = 1.5 or other values 
dq0 = 7.856742013183863e-5
mu=0.5
dmu = 0.00039283710065919316
delta = 0.0007071067811865476
for q in qs:
    dqs = np.linspace(2,7,10)*dq0
    ff = FFState(mu=mu, dmu=dmu, d=2, delta=delta, k_c=500)
    ps = [ff.get_pressure(mu=mu, dmu=dmu,q = q, dq=dq).n for dq in dqs]
    clear_output()
    plt.plot(dqs * 1000, ps)

# 3D Cases:
* the 3D cases are very sensitive to the cutoff k_c

### homogeneous density

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=3)

### Good accuracy

In [ ]:
test_Thermodynamic(mu = 5, dmu = 0.64, dim = 3, k_c = 250, q = 0, dq = 0)

### Poor density accurary

In [ ]:
test_Thermodynamic(mu = 5, dmu = 0.64, dim = 3, k_c = 100, q = 0, dq = 0)